In [1]:
from datasets import load_dataset
def load_data(max_length=512):
    print("Loading dataset from HuggingFace...")
    dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
    test_texts = dataset['test']['text']

    filtered_texts = [text for text in test_texts if text.strip() and len(text.split()) <= max_length and len(text.split()) > 50]
    print(f"Processed {len(filtered_texts)} samples after filtering")

    return filtered_texts

In [2]:
filtered_data = load_data()

Loading dataset from HuggingFace...
Processed 1626 samples after filtering


In [3]:
filtered_data[0]

' Robert Boulter is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John Deed in 2002 . In 2004 Boulter landed a role as " Craig " in the episode " Teddy \'s Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the Menier Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . \n'

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
llama_1b = AutoModelForCausalLM.from_pretrained('models/meta-llama/llama-3.2-1B').to('mps')
llama_3b = AutoModelForCausalLM.from_pretrained('models/meta-llama/llama-3.2-3B').to('mps')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
device = 'mps'

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

class LlamaIntermediateLayerExtractor:
    def __init__(self, model_3b_path):
        """
        Initialize extractor for Llama 3.2 1b and 3b models

        Args:
            model_1b_path (str): Path or HuggingFace model ID for 1b model
            model_3b_path (str): Path or HuggingFace model ID for 3b model
        """
        # Load models and tokenizers
        self.model_3b = AutoModelForCausalLM.from_pretrained(model_3b_path).to(device)

        self.tokenizer_1b = AutoTokenizer.from_pretrained("tokenizers/meta-llama/llama-3.2-1B")
        self.tokenizer_3b = AutoTokenizer.from_pretrained("tokenizers/meta-llama/llama-3.2-3B")

        self.model_3b.eval()

        # Hooks to capture intermediate layer outputs
        self.intermediate_output_3b = None


    def _register_3b_hook(self):
        """Register hook for 3b model's 18th layer"""
        def hook(module, input, output):
            self.intermediate_output_3b = output[0]

        # Adjust this path based on your specific model structure
        target_layer = self.model_3b.model.layers[17]  # 0-indexed, so 18th layer is index 17
        self.hook_3b = target_layer.register_forward_hook(hook)

    def extract_intermediate_representations(self, text_chunks, max_length=512):
        """
        Extract intermediate representations for given text chunks

        Args:
            text_chunks (list): List of text chunks to process
            max_length (int): Maximum token length to process

        Returns:
            tuple: (intermediate representations for 1b, intermediate representations for 3b)
        """
        # Reset intermediate outputs
        self.intermediate_output_3b = None

        self._register_3b_hook()

        repr_3b_list = []

        try:
            for chunk in text_chunks:

                # Tokenize and process 3b model
                inputs_3b = self.tokenizer_3b(
                    chunk,
                    return_tensors='pt',
                    truncation=True,
                    max_length=max_length
                ).to(device)

                # Forward pass to trigger hooks
                with torch.no_grad():
                    _ = self.model_3b(**inputs_3b)


                if self.intermediate_output_3b is not None:
                    repr_3b_list.append(self.intermediate_output_3b.detach())

        finally:
            self.hook_3b.remove()

        return repr_3b_list



In [7]:
extractor = LlamaIntermediateLayerExtractor('models/meta-llama/llama-3.2-3B')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
llama_1b_trunc = AutoModelForCausalLM.from_pretrained('models/meta-llama/llama-3.2-1B')

In [9]:
llama_1b_trunc.model.layers = llama_1b_trunc.model.layers[10:]

In [10]:
llama_1b_trunc.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-5): 6 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): L

In [11]:
import torch
import torch.nn as nn

class Seq2SeqEncoder(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, num_layers: int):
        """
        Encoder for seq2seq model.
        Args:
            input_dim: Dimensionality of input vectors
            hidden_dim: Hidden state size of the LSTM
            num_layers: Number of LSTM layers
        """
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

    def forward(self, x):
        """
        Args:
            x: Input tensor of shape (batch_size, seq_len, input_dim)
        Returns:
            encoder_outputs: Outputs for each time step (batch_size, seq_len, hidden_dim)
            hidden: Tuple of (h_n, c_n) (last hidden and cell states)
        """
        encoder_outputs, hidden = self.lstm(x)
        return encoder_outputs, hidden


class Seq2SeqDecoder(nn.Module):
    def __init__(self, hidden_dim: int, output_dim: int, num_layers: int):
        """
        Decoder for seq2seq model.
        Args:
            hidden_dim: Dimensionality of the encoded hidden state
            output_dim: Dimensionality of output vectors (same as input_dim)
            num_layers: Number of LSTM layers (should match encoder)
        """
        super().__init__()
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, hidden):
        """
        Args:
            x: Input tensor to the decoder (batch_size, seq_len, hidden_dim)
            hidden: Tuple of (h_n, c_n) from the encoder
        Returns:
            outputs: Decoded sequence (batch_size, seq_len, output_dim)
        """
        lstm_out, hidden = self.lstm(x, hidden)
        outputs = self.fc(lstm_out)
        return outputs, hidden


class Seq2SeqModel(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int, num_layers: int):
        """
        Combines encoder and decoder into a seq2seq model.
        Args:
            input_dim: Dimensionality of input vectors
            hidden_dim: Hidden state size
            num_layers: Number of LSTM layers
        """
        super().__init__()
        self.encoder = Seq2SeqEncoder(input_dim, hidden_dim, num_layers)
        self.decoder = Seq2SeqDecoder(hidden_dim, output_dim, num_layers)

    def forward(self, x):
        """
        Args:
            x: Input sequence of shape (batch_size, seq_len, input_dim)
        Returns:
            output: Reconstructed sequence of shape (batch_size, seq_len, input_dim)
        """
        # Encoder
        encoder_outputs, hidden = self.encoder(x)

        # Decoder (Use encoder outputs as initial input)
        output, _ = self.decoder(encoder_outputs, hidden)
        return output

In [12]:
class TransformerEmbedder(nn.Module):
    def __init__(self, input_dim, output_dim, layers):
        super(TransformerEmbedder, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=8, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=layers)
        self.activation = nn.GELU()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        mask = nn.Transformer().generate_square_subsequent_mask(x.shape[1])
        #print(mask.shape)
        x = self.transformer_encoder(x, mask=mask, is_causal=True)
        x = self.activation(x)
        x = self.fc(x)

        return x

In [13]:
import torch
import torch.nn as nn

class denseModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(denseModel, self).__init__()
        layers = [
            nn.Linear(input_size, hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, output_size)
        ]
        self.model = nn.Sequential(*layers)
    def forward(self, x):
        return self.model(x)



In [41]:
#seq2seqmodel_cos_1000 = torch.load('seq2seqmodel_1000_cosine.pth', weights_only=False)
#seq2seqmodel_1000 = torch.load('seq2seqmodel_1000.pth', weights_only=False)
#transformer1000 = torch.load('transformer_embedder_1000.pth', weights_only=False)
simple_encoder_cosine = torch.load('simple_encoder_decoder_3_layers_cosine.pth', weights_only=False).to(device)

In [42]:
tokenizer = AutoTokenizer.from_pretrained('tokenizers/meta-llama/llama-3.2-1B')

In [43]:
from tqdm import tqdm

In [26]:
simple_encoder_cosine = torch.load('simple_encoder_decoder_3_layers.pth', weights_only=False)

In [46]:
results = {
    'original': [],
    'llama_3_1': [],
    'dense_cosine': [],
    'llama_3_3': []
}
for text in tqdm(filtered_data[:100]):
    results['original'].append(text)
    with torch.no_grad():
        logits = llama_1b(input_ids = tokenizer.encode(text, return_tensors='pt').to(device)).logits
        output = tokenizer.decode(torch.argmax(logits, -1).squeeze())
        results['llama_3_1'].append(output)
        logits = llama_3b(input_ids = tokenizer.encode(text, return_tensors='pt').to(device)).logits
        output = tokenizer.decode(torch.argmax(logits, -1).squeeze())
        results['llama_3_3'].append(output)
        intermediate = extractor.extract_intermediate_representations([text])
        projected = simple_encoder_cosine(intermediate[0])
        output = llama_1b_trunc(inputs_embeds = projected)
        results['dense_cosine'].append(tokenizer.decode(torch.argmax(output.logits, -1).squeeze()))
        break




  0%|          | 0/100 [00:04<?, ?it/s]


In [28]:
from sentence_transformers import SentenceTransformer
sentences = [results['original'][0], results['llama_3_1'][0], results['dense_cosine'][0], results['llama_3_3'][0]]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)
print(embeddings)


[[-0.00477041 -0.02355691  0.0128001  ...  0.00318041 -0.01829152
  -0.00857943]
 [ 0.02149017  0.02279039  0.01305025 ...  0.0407836  -0.0451827
  -0.01794937]
 [ 0.01343295 -0.05770705  0.01454598 ...  0.03309499 -0.02617651
  -0.01864703]
 [ 0.02346542  0.00675755  0.01517578 ...  0.05409851 -0.03955113
  -0.01145439]]


In [38]:
results['llama_3_1'][1]

'Question the2009, the.ton was in heredaw in the film The, by David Ravenhill. The also in stage number2007 episode of the BBC series The The, as by a  in the 2007 film production of The to S a by Davidie Rourke. In to Curse was a at the Theatre, London West Borough of Hammersmith and Fulham. Inoulter appeared in the episodes in 2007, includingbreak Robbery and director David Barclesi and and the\'t Punch by by Davidly Blackburn. In  2008, Boulter appeared his guest appearance on the -part episode episode episode of of the BBC series,aking the Dead. playing by a appearance in the  series,ivors in  2008. In appeared a role role in the episodes of the television series Thety in 2000. and well Drieran " ". Inoulter appeared in the 2011 film Thei, by David Leonti, InIn'

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [39]:
llama_to_llama_sim = []
llama_to_dense_sim = []
for i in tqdm(range(100)):
    llama_3_1_embedding = model.encode([results['llama_3_1'][i]])
    dense_cosine_embedding = model.encode([results['dense_cosine'][i]])
    llama_3_3_embedding = model.encode([results['llama_3_3'][i]])
    llama_to_llama_sim.append(cosine_similarity(llama_3_1_embedding, llama_3_3_embedding))
    llama_to_dense_sim.append(cosine_similarity(llama_3_1_embedding, dense_cosine_embedding))


100%|██████████| 100/100 [00:06<00:00, 15.57it/s]


In [40]:
print(np.mean(llama_to_llama_sim))
print(np.mean(llama_to_dense_sim))

0.84712243
0.81373733


In [29]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

print(cosine_similarity(np.array([embeddings[1]]), np.array([embeddings[2]])))
print(cosine_similarity(np.array([embeddings[1]]), np.array([embeddings[3]])))

[[0.77080214]]
[[0.9091305]]


In [42]:
results = {
    'original': [],
    'llama_3_1': [],
    'llama_3_3_to_1_seq2seqcosine': [],
    'llama_3_3_to_1_seq2seq': [],
    'llama_3_3_to_1_transformer': []
}
for text in tqdm(filtered_data):
    results['original'].append(text)
    with torch.no_grad():
        logits = llama_1b(input_ids = tokenizer.encode(text, return_tensors='pt').to(device)).logits
        output = tokenizer.decode(torch.argmax(logits, -1).squeeze())
        results['llama_3_1'].append(output)
        intermediate = extractor.extract_intermediate_representations([text])

        projected_seq2seq_cosine = seq2seqmodel_cos_1000(intermediate[0])
        projected_seq2seq = seq2seqmodel_1000(intermediate[0])
        projected_transformer = transformer1000(intermediate[0])

        logits_seq2seq_cosine = llama_1b_trunc(inputs_embeds = projected_seq2seq_cosine).logits
        logits_seq2seq = llama_1b_trunc(inputs_embeds = projected_seq2seq).logits
        logits_transformer = llama_1b_trunc(inputs_embeds = projected_transformer).logits

        output_seq2seq_cosine = tokenizer.decode(torch.argmax(logits_seq2seq_cosine, -1).squeeze())
        output_seq2seq = tokenizer.decode(torch.argmax(logits_seq2seq, -1).squeeze())
        output_transformer = tokenizer.decode(torch.argmax(logits_transformer, -1).squeeze())

        results['llama_3_3_to_1_seq2seqcosine'].append(output_seq2seq_cosine)
        results['llama_3_3_to_1_seq2seq'].append(output_seq2seq)
        results['llama_3_3_to_1_transformer'].append(output_transformer)




100%|██████████| 1626/1626 [41:14<00:00,  1.52s/it]  


In [43]:
import json
with open("seq2seq_results.json", "w") as f:
    json.dump(results, f)

In [2]:
import json
with open("seq2seq_results.json", "r") as f:
    results = json.load(f)

In [1]:
random_item = 1000
print(results['original'][random_item])
print(results['llama_3_1'][random_item])
print(results['llama_3_3_to_1_seq2seqcosine'][random_item])
print(results['llama_3_3_to_1_seq2seq'][random_item])
print(results['llama_3_3_to_1_transformer'][random_item])

NameError: name 'results' is not defined

In [38]:
import torch
import torch.nn as nn

class SimpleEncoderDecoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleEncoderDecoder, self).__init__()
        self.encoder = nn.Linear(input_dim, hidden_dim)
        self.decoder = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.encoder(x)
        x = self.relu(x)
        x = self.decoder(x)
        return x

In [39]:
encoder_decoder = torch.load('encoder_decoder_3b_1b_1000.pth', weights_only=False)

In [40]:
projected = encoder_decoder(a[0])

In [41]:
projected.shape

torch.Size([1, 193, 2048])

In [42]:
projected

tensor([[[ 0.8522, -0.9071,  0.3003,  ...,  1.7527,  1.0636,  0.2268],
         [-0.0397,  0.0355,  0.1739,  ..., -0.0112, -0.1490, -0.0518],
         [-0.0396, -0.0071,  0.3657,  ..., -0.0252, -0.0199, -0.0686],
         ...,
         [-0.0035, -0.0020,  0.1780,  ..., -0.0967,  0.0351, -0.0145],
         [ 0.0417,  0.0149,  0.0511,  ..., -0.0183, -0.0489, -0.0158],
         [ 0.0794, -0.0185, -0.0200,  ...,  0.0145,  0.0353, -0.0233]]],
       device='mps:0', grad_fn=<LinearBackward0>)

In [43]:
output = llama_1b_trunc(inputs_embeds = projected)

In [44]:
output.logits.shape

torch.Size([1, 193, 128256])

In [45]:
torch.argmax(output.logits, -1).shape

torch.Size([1, 193])

In [46]:
tokenizer = AutoTokenizer.from_pretrained('tokenizers/meta-llama/llama-3.2-1B')

In [47]:
tokenizer.decode(torch.argmax(output.logits, -1).squeeze())

'Question and.man\n a former musician producer and artist singer artist. He is been number appearance worktour- role on the  series The New. 2006. He was the in a solo role on the album Theland. by the H. which was released in 2006. the band Society.. He has a solo appearance on the  series The John Steeleering, 2009. He 2009,@200 was a solo in the the\'in the film, The\'ss " " of the film series The Rolling Walk. he also the the and St and Steve Fey. In also also in the film2009 album album of the album and album The.. which was released in the Royal video in . the\'s magazine.. . In also also by the D, was the the andiganam. and andey, and De, and.er, and and, Steve L. HeIn'

In [21]:
tokenized_q = tokenizer(input, return_tensors='pt')
out = llama_1b(**tokenized_q)

In [22]:
out.logits.shape

torch.Size([1, 193, 128256])

In [23]:
tokenizer.decode(torch.argmax(out.logits, -1).squeeze())

'Question E.ton\n a American actor and television and theatre actor. He is a long role rolerole role role in the BBC series The Bill in 1980. He was his by a role role in the  Theod in by David Gray. which was performed at the2001 at the Royal Court Theatre in He has a guest role in the television series The John Deed in 2002. He 2003 heoulter played a role in the Mr " in the television " The Bears Day " of the television series The Bill Firm. this also in the such Strong, James Jacobi. In also also as the role2005 film production of The play Ridley play The Fur and and was performed at the Royal Theatre in London, at Royalier Chocolate Factory in London. In played cast by the Tiffany. starred alongside actors Milesishaw, David Richieaza and and Lloyd and and Jamesres and and Thompson, David Row. HeRobert'